In [1]:
import gurobipy as gp

# Create a new model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02


In [2]:
# Initialize problem parameters
scraps = 3
boxes = 6

set_scraps = range(0, scraps)
set_boxes = range(0, boxes + 1)

# Box distance matrix
d = [[0, 1, 2, 3, 4, 5, 6],
    [1, 0, 1, 2, 3, 4, 5],
    [2, 1, 0, 1, 2, 3, 4],
    [3, 2, 1, 0, 1, 2, 3],
    [4, 3, 2, 1, 0, 1, 2],
    [5, 4, 3, 2, 1, 0, 1],
    [6, 5, 4, 3, 2, 1, 0]]

# Distance from hub
d_hub = [0, 1, 2, 3, 4, 5, 6]

# Distance from end
d_end = [6, 5, 4, 3, 2, 1, 0]

recipes = [[0, 1], [1, 2], [2, 0]]

In [3]:
# Create variables

# Assign scraps to boxes
x = {}
for scrap in set_scraps:
    for box in set_boxes:
        x[scrap, box] = m.addVar(vtype='B', name=f"x[{scrap},{box}]")

# Distance from hub to scrap
d_hub_var = {}
for scrap in set_scraps:
    for box in set_boxes:
        d_hub_var[scrap, box] = m.addVar(vtype='I', name=f"d_end[{scrap},{box}]")

# Minimum distance from hub to scrap type
x_min = {}
for scrap in set_scraps:
    x_min[scrap] = m.addVar(vtype='I', name=f"x_min[{scrap}]")



In [4]:
# Add constraints

# Each scrap is assigned to at least one box
for s in set_scraps:
    m.addConstr(sum(x[s, b] for b in set_boxes) >= 1)

# Each box is assigned at most one scrap
for b in set_boxes[1:]:
    m.addConstr(sum(x[s, b] for s in set_scraps) == 1)

# Dummy box is empty
m.addConstr(sum(x[s, 0] for s in set_scraps) == 0)

# D_hub_var is the distance from the hub
for s in set_scraps:
    for b in set_boxes[1:]:
        m.addConstr(d_hub_var[s, b] == boxes - d_end[b] * x[s, b])

# X_min is the minimum distance from the hub
for s in set_scraps:
    m.addGenConstrMin(x_min[s], [d_hub_var[s, b] for b in set_boxes[1:]], name=f"x_min[{s}]")


In [5]:
# Set objective function

# Test
#objective = gp.quicksum(x[1, b] for b in set_boxes)

# Simplified objective
objective = gp.quicksum(gp.quicksum(x_min[s] for s in recipe) for recipe in recipes)

m.setObjective(objective, gp.GRB.MINIMIZE)

In [6]:
# Solve it!
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 PRO 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 28 rows, 45 columns and 75 nonzeros
Model fingerprint: 0xca7105dc
Model has 3 general constraints
Variable types: 0 continuous, 45 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve added 14 rows and 6 columns
Presolve time: 0.00s
Presolved: 42 rows, 51 columns, 141 nonzeros
Variable types: 0 continuous, 51 integer (33 binary)
Found heuristic solution: objective 12.0000000

Root relaxation: objective 9.384615e+00, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumb

In [7]:
# Show solution
print(f"Optimal objective value: {m.objVal}")

for v in m.getVars():
    if v.X > 0:
        print(f"{v.VarName} = {v.X}")

Optimal objective value: 12.0
x[0,1] = 1.0
x[1,2] = 1.0
x[2,3] = 1.0
x[2,4] = 1.0
x[2,5] = 1.0
x[2,6] = 1.0
d_end[0,1] = 1.0
d_end[0,2] = 6.0
d_end[0,3] = 6.0
d_end[0,4] = 6.0
d_end[0,5] = 6.0
d_end[0,6] = 6.0
d_end[1,1] = 6.0
d_end[1,2] = 2.0
d_end[1,3] = 6.0
d_end[1,4] = 6.0
d_end[1,5] = 6.0
d_end[1,6] = 6.0
d_end[2,1] = 6.0
d_end[2,2] = 6.0
d_end[2,3] = 3.0
d_end[2,4] = 4.0
d_end[2,5] = 5.0
d_end[2,6] = 6.0
x_min[0] = 1.0
x_min[1] = 2.0
x_min[2] = 3.0
